In [97]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [98]:
#Special Occasions
#Rivalries
#After a loss
#Betting Odds


In [99]:
df = pd.read_csv(r'C:\Users\raadr\OneDrive\Desktop\AflAnalysis-\data\interim\fullSesData.csv')
df

,GameId,Year,Round,Date,MaxTemp,MinTemp,Rainfall,Venue,StartTime,Attendance,...,homePoints,homePercentage,awayPosition,awayPoints,awayPercentage,Home Odds,Away Odds,Day,Month,MinutesSinceMidnight
0,2012R0102,2012,1,2012-03-29,25.7,9.7,0.0,1,1900-01-01 19:45:00,78285,...,0.0,64.8,4.0,4.0,154.3,2.48,1.52,3,3,1185
1,2012R0103,2012,1,2012-03-30,27.4,9.7,0.0,1,1900-01-01 19:50:00,78466,...,4.0,119.1,13.0,0.0,83.9,1.65,2.19,4,3,1190
2,2012R0104,2012,1,2012-03-31,29.1,15.1,0.6,1,1900-01-01 13:45:00,33473,...,0.0,65.5,5.0,4.0,152.6,1.42,2.81,5,3,825
3,2012R0105,2012,1,2012-03-31,28.2,19.7,0.0,2,1900-01-01 15:45:00,12790,...,0.0,49.6,2.0,4.0,201.5,4.39,1.22,5,3,945
4,2012R0106,2012,1,2012-03-31,24.0,9.0,0.0,3,1900-01-01 16:45:00,34601,...,4.0,104.0,11.0,0.0,96.2,1.96,1.79,5,3,1005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1944,2023SF01,2023,-3,2023-09-15,24.5,14.4,0.0,1,1900-01-01 19:50:00,96412,...,60.0,124.8,5.0,54.0,116.1,1.61,2.21,4,9,1190
1945,2023SF02,2023,-3,2023-09-16,24.0,7.6,0.0,11,1900-01-01 19:10:00,45520,...,68.0,112.7,7.0,52.0,107.1,1.62,2.18,5,9,1150
1946,2023PF01,2023,-2,2023-09-22,15.3,5.7,0.0,1,1900-01-01 19:50:00,97665,...,68.0,123.2,7.0,52.0,107.1,1.55,2.34,4,9,1190
1947,2023PF02,2023,-2,2023-09-23,24.1,14.2,0.0,6,1900-01-01 17:15:00,36012,...,68.0,123.1,5.0,54.0,116.1,1.38,3.05,5,9,1035


In [100]:
import pandas as pd

# Suppose your original DataFrame is called df and has columns like:
# 'Date', 'HomeTeam', 'awayTeam', 'homePosition', 'homePoints', 'homePercentage',
# 'awayPosition', 'awayPoints', 'awayPercentage'
#
# (Adjust the column names if needed.)

# Step 1: Sort the DataFrame by date
df = df.sort_values(by='Date', ascending=True)

# Step 2: Build a unified team-games DataFrame from the home and away appearances.
# Extract home team appearances:
df_home = df[['Date', 'HomeTeam', 'homePosition', 'homePoints', 'homePercentage']].copy()
df_home = df_home.rename(columns={
    'HomeTeam': 'Team',
    'homePosition': 'position',
    'homePoints': 'points',
    'homePercentage': 'percentage'
})

# Extract away team appearances:
df_away = df[['Date', 'AwayTeam', 'awayPosition', 'awayPoints', 'awayPercentage']].copy()
df_away = df_away.rename(columns={
    'AwayTeam': 'Team',
    'awayPosition': 'position',
    'awayPoints': 'points',
    'awayPercentage': 'percentage'
})

# Concatenate both so that every game for every team is in one DataFrame.
df_team = pd.concat([df_home, df_away], ignore_index=True)

# Sort by Team and Date so that each team’s games are in chronological order.
df_team = df_team.sort_values(by=['Team', 'Date'], ascending=True)

# Step 3: For each team, shift the stats to create previous game columns.
df_team['prev_position'] = df_team.groupby('Team')['position'].shift(1)
df_team['prev_points'] = df_team.groupby('Team')['points'].shift(1)
df_team['prev_percentage'] = df_team.groupby('Team')['percentage'].shift(1)

# Now, df_team has, for every game a team played, the stats of its previous game.
# The first game for each team will naturally have NaN in these new columns.

# Step 4: Merge the previous game info back into the original DataFrame.
# Since the same game date appears in df_team as in df for that team’s game,
# we can merge on both Date and Team.

# For the home team info:
# We need to merge df with the corresponding rows in df_team for when Team == HomeTeam.
home_prev = df_team[['Date', 'Team', 'prev_position', 'prev_points', 'prev_percentage']].copy()
home_prev = home_prev.rename(columns={
    'Team': 'HomeTeam',
    'prev_position': 'previous_game_home_position',
    'prev_points': 'previous_game_home_points',
    'prev_percentage': 'previous_game_home_percentage'
})

# Merge on Date and HomeTeam
df = pd.merge(df, home_prev, on=['Date', 'HomeTeam'], how='left')

# For the away team info:
away_prev = df_team[['Date', 'Team', 'prev_position', 'prev_points', 'prev_percentage']].copy()
away_prev = away_prev.rename(columns={
    'Team': 'AwayTeam',  # Note: use the same case as your original df column
    'prev_position': 'previous_game_away_position',
    'prev_points': 'previous_game_away_points',
    'prev_percentage': 'previous_game_away_percentage'
})
# Merge on Date and awayTeam
df = pd.merge(df, away_prev, on=['Date', 'AwayTeam'], how='left')


In [101]:
df[['HomeTeam', 'AwayTeam', 'previous_game_home_position', 'previous_game_away_position']].iloc[1:15]

,HomeTeam,AwayTeam,previous_game_home_position,previous_game_away_position
1,10,4,NaN,NaN
2,11,2,NaN,NaN
3,8,1,NaN,NaN
4,6,7,NaN,NaN
5,12,5,NaN,NaN
6,18,17,NaN,NaN
7,13,15,NaN,NaN
8,2,3,5.0,4.0
9,5,13,9.0,7.0
10,16,6,NaN,8.0
